In [6]:
from modules import resnet1,trainer
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,random_split
from torch.nn import CrossEntropyLoss

In [7]:
device=torch.device('cuda')
def fit(model:resnet1.ResNet, optim,lossfunc,trainloader: DataLoader):
    model.train()
    totalloss = 0
    for data, target in trainloader:
        data, target = data.to(device), target.to(device)
        optim.zero_grad()
        output = model(data)
        loss = lossfunc(output, target)
        with torch.no_grad():
            totalloss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(),3.0)
        optim.step()
    return totalloss

def evaluate(model:ResNet.ResNet20, val_loader: DataLoader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [8]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='/d/usr14/project/Binary/wwdata', train=True,
                                        download=False, transform=transform)
train_size = 45000
val_size = 5000
train_dataset, val_dataset = random_split(trainset, [train_size, val_size])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128,
                                          shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)

testset = torchvision.datasets.CIFAR10(root='/d/usr14/project/Binary/wwdata', train=False,
                                       download=False, transform=transform)

test_loader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [9]:
bw=ba=False
print(bw, ba)
savepath = "D:/usr14/project/Binary/wwdata/cifa10/fullbest.pth"
if bw and not ba:
    mode = "w"
elif ba and not bw:
    mode = "a"
else:
    mode = "b"
mode="n"
print("mnist, mode: " + mode)
model = ResNet.ResNet20(bw, ba).to(device)
lossfunc=nn.CrossEntropyLoss()

False False
mnist, mode: n


In [10]:
lr=1
for epoch in range(80):
    if epoch%10==0:
        lr/=10
        optim=torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    loss=fit(model,optim,lossfunc,train_loader)
    tacc=evaluate(model,train_loader)
    acc=evaluate(model,val_loader)
    print(f'epoch: {epoch}, loss: {loss}, train_acc:{tacc}, val_acc:{acc}')
torch.save(model.state_dict(), 'pre1.pth')

epoch: 0, loss: 527.3771959543228, train_acc:0.5704222222222223, val_acc:0.5734
epoch: 1, loss: 380.9333062171936, train_acc:0.6572222222222223, val_acc:0.6526
epoch: 2, loss: 332.5422064065933, train_acc:0.6764, val_acc:0.6646
epoch: 3, loss: 307.262766122818, train_acc:0.7109111111111112, val_acc:0.6896
epoch: 4, loss: 282.11122596263885, train_acc:0.719, val_acc:0.701
epoch: 5, loss: 265.2082188129425, train_acc:0.7427333333333334, val_acc:0.7228
epoch: 6, loss: 255.88003078103065, train_acc:0.763, val_acc:0.7436
epoch: 7, loss: 240.11122804880142, train_acc:0.7549555555555556, val_acc:0.7302
epoch: 8, loss: 232.33611372113228, train_acc:0.7801777777777777, val_acc:0.754
epoch: 9, loss: 222.4561694264412, train_acc:0.8011111111111111, val_acc:0.7714
epoch: 10, loss: 158.4303584396839, train_acc:0.8647111111111111, val_acc:0.8194
epoch: 11, loss: 141.23211106657982, train_acc:0.8769333333333333, val_acc:0.8266
epoch: 12, loss: 133.71970984339714, train_acc:0.8825555555555555, val_acc